In [88]:
import tensorflow as tf
import numpy as np
from tensorflow import keras
from random import shuffle

In [2]:
temp_min = np.load("Temperature_min.npy")

In [3]:
len(temp_min)

25551

In [4]:
len(temp_min)*.75,len(temp_min)*.15,len(temp_min)*.10

(19163.25, 3832.6499999999996, 2555.1000000000004)

In [5]:
temp_min_train, temp_min_valid, temp_min_test = temp_min[:19163], temp_min[19163:19163+3832], temp_min[19163+3832:]

In [6]:
len(temp_min_train), len(temp_min_valid), len(temp_min_test)

(19163, 3832, 2556)

I will now break the data into windows

In [167]:
def create_target(batch):
    X = batch[ :, :-1] #first elements for X
    y = batch[ :, 1:] # last elements for y
    return X, y

In [170]:
def batch_dataset(data, window_size=50, window_shift=25):
    data_set = []
    i=0
    while window_shift*i+window_size<len(data)+1:
        window = data[window_shift*i:window_shift*i+window_size]
        data_set.append(np.array(window))
        i+=1
    shuffle(data_set)
    return np.array(data_set).reshape(len(data_set),window_size,1)

In [171]:
batched_temp_min_train = batch_dataset(temp_min_train)
batched_temp_min_valid = batch_dataset(temp_min_valid)
batched_temp_min_test = batch_dataset(temp_min_test)

In [172]:
batched_temp_min_train.shape

(765, 50, 1)

In [173]:
X_train, y_train = create_target(batched_temp_min_train)
X_valid, y_valid = create_target(batched_temp_min_valid)
X_test, y_test = create_target(batched_temp_min_test)

In [174]:
X_train[0]

array([[0.27953088],
       [0.72933996],
       [1.29160131],
       [0.84179223],
       [1.29160131],
       [0.84179223],
       [0.39198315],
       [1.29160131],
       [1.74141039],
       [1.17914904],
       [0.72933996],
       [0.50443542],
       [0.61688769],
       [0.27953088],
       [0.05462634],
       [0.16707861],
       [0.72933996],
       [0.61688769],
       [0.50443542],
       [0.72933996],
       [0.50443542],
       [1.06669677],
       [1.17914904],
       [1.17914904],
       [1.06669677],
       [1.29160131],
       [1.17914904],
       [1.17914904],
       [1.06669677],
       [1.29160131],
       [1.96631494],
       [1.62895812],
       [1.40405358],
       [1.17914904],
       [1.17914904],
       [1.17914904],
       [1.17914904],
       [1.29160131],
       [1.29160131],
       [1.51650585],
       [1.85386266],
       [1.17914904],
       [1.17914904],
       [1.40405358],
       [1.51650585],
       [1.06669677],
       [1.40405358],
       [1.404

In [175]:
y_train[0]

array([[0.72933996],
       [1.29160131],
       [0.84179223],
       [1.29160131],
       [0.84179223],
       [0.39198315],
       [1.29160131],
       [1.74141039],
       [1.17914904],
       [0.72933996],
       [0.50443542],
       [0.61688769],
       [0.27953088],
       [0.05462634],
       [0.16707861],
       [0.72933996],
       [0.61688769],
       [0.50443542],
       [0.72933996],
       [0.50443542],
       [1.06669677],
       [1.17914904],
       [1.17914904],
       [1.06669677],
       [1.29160131],
       [1.17914904],
       [1.17914904],
       [1.06669677],
       [1.29160131],
       [1.96631494],
       [1.62895812],
       [1.40405358],
       [1.17914904],
       [1.17914904],
       [1.17914904],
       [1.17914904],
       [1.29160131],
       [1.29160131],
       [1.51650585],
       [1.85386266],
       [1.17914904],
       [1.17914904],
       [1.40405358],
       [1.51650585],
       [1.06669677],
       [1.40405358],
       [1.40405358],
       [1.291

In [176]:
X_train.shape,y_train.shape

((765, 49, 1), (765, 49, 1))

All good! Time to train! I will start a group of convolutions,2 GRUs for long term and and output dense with no activation

In [177]:
model = keras.models.Sequential([
    keras.layers.Conv1D(32, kernel_size=2, padding="causal", activation="relu"),
    keras.layers.BatchNormalization(),
    keras.layers.Conv1D(48, kernel_size=2, padding="causal", activation="relu", dilation_rate=2),
    keras.layers.BatchNormalization(),
    keras.layers.Conv1D(64, kernel_size=2, padding="causal", activation="relu", dilation_rate=4),
    keras.layers.BatchNormalization(),
    keras.layers.Conv1D(96, kernel_size=2, padding="causal", activation="relu", dilation_rate=8),
    keras.layers.BatchNormalization(),
    keras.layers.GRU(20, return_sequences = True), 
    keras.layers.GRU(20, return_sequences = True), 
    keras.layers.TimeDistributed( keras.layers.Dense(1, activation = None)) ])

In [178]:
early_stopping_cb = keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)

In [179]:
model.compile( loss ="mse", optimizer ="adam") 

In [180]:
history = model.fit( X_train, y_train, epochs = 1000, validation_data =( X_valid, y_valid),callbacks=[early_stopping_cb])

Train on 765 samples, validate on 152 samples
Epoch 1/1000
765/765 [==============================] - 14s 19ms/sample - loss: 0.4199 - val_loss: 0.7865
Epoch 2/1000
765/765 [==============================] - 3s 4ms/sample - loss: 0.2311 - val_loss: 0.7610
Epoch 3/1000
765/765 [==============================] - 3s 4ms/sample - loss: 0.2030 - val_loss: 0.7041
Epoch 4/1000
765/765 [==============================] - 3s 4ms/sample - loss: 0.1966 - val_loss: 0.6224
Epoch 5/1000
765/765 [==============================] - 3s 4ms/sample - loss: 0.2004 - val_loss: 0.6437
Epoch 6/1000
765/765 [==============================] - 3s 4ms/sample - loss: 0.1894 - val_loss: 0.5379
Epoch 7/1000
765/765 [==============================] - 4s 5ms/sample - loss: 0.1830 - val_loss: 0.5362
Epoch 8/1000
765/765 [==============================] - 4s 5ms/sample - loss: 0.1803 - val_loss: 0.4782
Epoch 9/1000
765/765 [==============================] - 3s 4ms/sample - loss: 0.1787 - val_loss: 0.4253
Epoch 10/1000
76

In [181]:
model.evaluate(X_test,y_test)

101/101 [==============================] - 0s 1ms/sample - loss: 0.1605


0.16054220143521186

In [182]:
pred = model.predict(X_test)

Great!

In [183]:
np.save("Min_Temp_pred_2",pred)
np.save("Min_Temp_y_2", y_test)